In [10]:
 !pip install nltk

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [107]:
!pip install -U sentence-transformers

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 8.1 MB/s eta 0:00:00


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import re
from keras.models import Sequential
from keras.layers import Input, Embedding,TimeDistributed, LSTM, Dropout, Bidirectional, Dense 
import random
from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.utils import plot_model, to_categorical
import nltk
from tqdm import tqdm
import subprocess

# Download and unzip wordnet
try:
    nltk.data.find('wordnet.zip')
except:
    nltk.download('wordnet', download_dir='/kaggle/working/')
    command = "unzip /kaggle/working/corpora/wordnet.zip -d /kaggle/working/corpora"
    subprocess.run(command.split())
    nltk.data.path.append('/kaggle/working/')

# Now you can import the NLTK resources as usual
from nltk.corpus import wordnet
nltk.download('punkt')


[nltk_data] Downloading package wordnet to /kaggle/working/...
Archive:  /kaggle/working/corpora/wordnet.zip
   creating: /kaggle/working/corpora/wordnet/
  inflating: /kaggle/working/corpora/wordnet/lexnames  
  inflating: /kaggle/working/corpora/wordnet/data.verb  
  inflating: /kaggle/working/corpora/wordnet/index.adv  
  inflating: /kaggle/working/corpora/wordnet/adv.exc  
  inflating: /kaggle/working/corpora/wordnet/index.verb  
  inflating: /kaggle/working/corpora/wordnet/cntlist.rev  
  inflating: /kaggle/working/corpora/wordnet/data.adj  
  inflating: /kaggle/working/corpora/wordnet/index.adj  
  inflating: /kaggle/working/corpora/wordnet/LICENSE  
  inflating: /kaggle/working/corpora/wordnet/citation.bib  
  inflating: /kaggle/working/corpora/wordnet/noun.exc  
  inflating: /kaggle/working/corpora/wordnet/verb.exc  
  inflating: /kaggle/working/corpora/wordnet/README  
  inflating: /kaggle/working/corpora/wordnet/index.sense  
  inflating: /kaggle/working/corpora/wordnet/data.

True

In [108]:
def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

In [109]:
from sentence_transformers import SentenceTransformer


embidding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')



modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [105]:
train_data = pd.read_json("/kaggle/input/pizza-dataset/PIZZA_train.json", lines=True)
train_data = train_data.sample(frac=0.01, random_state=42)
src=train_data["train.SRC"].to_numpy()
top_decoupled=train_data["train.TOP"].to_numpy()


In [ ]:
import logging
logging.getLogger('sentence_transformers').setLevel(logging.ERROR)
src_emd =[]
count=0
new_src=np.array([])
new_top_decoupled=np.array([])
for src_sent,top_sent in tqdm(zip(src,top_decoupled)):
    flag=False
    sent_embedding = embidding_model.encode(src_sent)
    for emd in src_emd:
        cos=cosine(emd,sent_embedding)
        # print(cos)
        if(cos>0.9):
            flag=True
            break
    if(not flag):
        count+=1
        src_emd.append(sent_embedding)
        new_src=np.append(new_src,src_sent)
        new_top_decoupled=np.append(new_top_decoupled,top_sent)

14215it [05:08, 37.96it/s]

In [25]:
# file_path = "src.txt"

# # Save to file
# with open(file_path, "w") as file:
#     for line in new_src:
#         file.write(line + "\n")

# file_path = "top_decoupled.txt"

# # Save to file
# with open(file_path, "w") as file:
#     for line in new_top_decoupled:
#         file.write(line + "\n")

In [26]:
# print("count =",count)

In [12]:
# file_path = "/kaggle/input/dataset/src.txt"

# # Read data into an array of strings
# with open(file_path, "r") as file:
#     new_src = np.array([line.strip() for line in file])

# file_path = "/kaggle/input/dataset/top_decoupled.txt"

# # Read data into an array of strings
# with open(file_path, "r") as file:
#     new_top_decoupled = np.array([line.strip() for line in file])

In [35]:
def preprosses(src,top_decoupled):
    word_tag=[]
    for src,trg in  zip(src,top_decoupled):
        out=extract_bio_tags(src,trg)
        word_tag.append(out)
    return word_tag




In [36]:
def get_pizza_order_synonyms(word):
    """
    Get synonyms related to the 'pizza order' category.
    """
    # Keywords related to pizza ordering
    pizza_keywords = [
        "pizza", "cheese", "crust", "pepperoni", "mushroom", 
        "topping", "delivery", "order","drink","meat","chicken","size","pizza style","negation"
    ]
    
    synonyms = set()
    for syn in wordnet.synsets(word):
        # Check if the synset definition matches pizza-related keywords
        if any(keyword in syn.definition().lower() for keyword in pizza_keywords):
            for lemma in syn.lemmas():
                synonym = lemma.name().replace("_", " ").replace("-", " ").lower()
                synonym = "".join([char for char in synonym if char in ' qwertyuiopasdfghjklzxcvbnm'])
                synonyms.add(synonym)
    
    # Remove the input word from synonyms if it exists
    if word in synonyms:
        synonyms.remove(word)
    
    return list(synonyms)

In [37]:
def synonym_replacement(sentence,top_decoupled, num_replacements=1):
    words = sentence.split()  # Tokenize sentence
    words_to_replace = [word for word in words if get_pizza_order_synonyms(word)]  # Only consider words with synonyms

    # Shuffle the list to randomly select words for replacement
    random.shuffle(words_to_replace)

    num_replaced = 0
    for word in words_to_replace:
        if num_replaced >= num_replacements:
            break
        
        synonyms = get_pizza_order_synonyms(word)
        if synonyms:
            synonym = random.choice(synonyms)  # Pick a random synonym
            words = [synonym if w == word else w for w in words]
            num_replaced += 1
            top_decoupled=re.sub(word,synonym,top_decoupled)

    return " ".join(words),top_decoupled

In [38]:
src=new_src
top_decoupled=new_top_decoupled
print(src.shape)
src_augment=src
top_decoupled_augment=top_decoupled
i=0
for src_itr,top_itr in tqdm(zip(src_augment,top_decoupled_augment)):
    srcc,topp=synonym_replacement(src_itr,top_itr,5)
    src=np.append(src,srcc)
    top_decoupled=np.append(top_decoupled,topp)
   
    

print(src.shape)

    
    
    


In [39]:
# file_path = "src_65.txt"

# # Save to file
# with open(file_path, "w") as file:
#     for line in src:
#         file.write(line + "\n")

# file_path = "top_decoupled_65.txt"

# # Save to file
# with open(file_path, "w") as file:
#     for line in top_decoupled:
#         file.write(line + "\n")

In [40]:
# file_path = "/kaggle/input/dataset65/src_65.txt"

# # Read data into an array of strings
# with open(file_path, "r") as file:
#     src = np.array([line.strip() for line in file])

# file_path = "/kaggle/input/dataset65/top_decoupled_65.txt"

# # Read data into an array of strings
# with open(file_path, "r") as file:
#     top_decoupled = np.array([line.strip() for line in file])

In [87]:
def extract_bio_tags(src,top_decoupled):
    # Parse the TOP-DECOUPLED structure
    matches = re.split(r"(?=\(PIZZAORDER|\(DRINKORDER)", top_decoupled)
    start=matches[0].split()
    start=start[1:]
    matches=matches[1:]
    start_index=len(start)
    
    
    ind=-1
    if random.random() < 0.5:
         ind=random.randint(0, len(src.split())-1)
    
    # Map words to BIO tags
    bio_tag_map = {}
    src_words = src.split()
    
    for i,match in enumerate(matches):
        
        match=re.sub(r'\(|\)','',match)
        match=match.strip()
        words= match.split()  # Split multi-word values
        tag=words[0]
        words=words[1:]
        for i, word in enumerate(words):
            if i == 1:
                if word in src_words[start_index:]:
                    bio_tag_map[src_words.index(word,start_index)]= f"B-{tag}"  # Begin tag
                    start_index+=1
            else:
                if word in src_words[start_index:]:
                    bio_tag_map[src_words.index(word,start_index)]= f"I-{tag}"  # Inside tag
                    start_index+=1
   
                
            
  
    
    word_tag_pairs = [(word, bio_tag_map.get(i, "O")) for i, word in enumerate(src_words)]
 
    if ind!=-1:
        word_tag_pairs[ind]=("<UNK>",word_tag_pairs[ind][1])
    
    return word_tag_pairs
 
 # word_bio_tag_pairs = extract_bio_tags(top_decoupled, src
# out=extract_bio_tags("can i get five small pizzas with mushroom and bacon and then a pepsi","(ORDER can i get (PIZZAORDER (NUMBER five ) (SIZE small ) pizzas with (TOPPING mushroom ) and (TOPPING bacon ) ) and then (DRINKORDER (NUMBER a ) (DRINKTYPE pepsi ) ) )")
# print(out)





In [88]:
sentences =preprosses(src,top_decoupled)


In [104]:
print(sentences[:20])

[[("i'd", 'O'), ('like', 'O'), ('three', 'B-PIZZAORDER'), ('large', 'I-PIZZAORDER'), ('pies', 'O'), ('with', 'O'), ('pestos', 'I-PIZZAORDER'), ('and', 'O'), ('yellow', 'I-PIZZAORDER'), ('peppers', 'I-PIZZAORDER')], [("i'd", 'O'), ('like', 'O'), ('a', 'B-PIZZAORDER'), ('small', 'I-PIZZAORDER'), ('pizza', 'O'), ('with', 'O'), ('pineapple', 'I-PIZZAORDER'), ('buffalo', 'I-PIZZAORDER'), ('chicken', 'I-PIZZAORDER'), ('and', 'O'), ('garlic', 'I-PIZZAORDER'), ('powder', 'I-PIZZAORDER')], [('three', 'B-PIZZAORDER'), ('party', 'I-PIZZAORDER'), ('sized', 'I-PIZZAORDER'), ('pizzas', 'O'), ('with', 'O'), ('<UNK>', 'I-PIZZAORDER'), ('and', 'O'), ('hot', 'I-PIZZAORDER'), ('pepper', 'I-PIZZAORDER')], [('balsamic', 'B-PIZZAORDER'), ('glaze', 'I-PIZZAORDER'), ('and', 'O'), ('a', 'B-DRINKORDER'), ('<UNK>', 'I-DRINKORDER'), ('and', 'O'), ('three', 'B-DRINKORDER'), ('eight', 'I-DRINKORDER'), ('ounce', 'I-DRINKORDER'), ('fantas', 'I-DRINKORDER'), ('and', 'O'), ('three', 'B-DRINKORDER'), ('sprites', 'I-DRIN

In [89]:


maxlen = max([len(s) for s in sentences])
words = [w[0] for sentence in sentences for w in sentence]
words = list(set(words))
words.append("<pad>")
n_words = len(words)
print('Number of unique words:', n_words)


Number of unique words: 346


In [90]:
# print(words)

In [91]:


tags = [w[1] for sentence in sentences for w in sentence]
tags=list(set(tags))
# tags.append("<pad>")
print(tags)
n_tags = len(tags)
print('Number of unique Tags:', n_tags)

['O', 'B-DRINKORDER', 'B-PIZZAORDER', 'I-DRINKORDER', 'I-PIZZAORDER']
Number of unique Tags: 5


In [92]:
word2idx = {w: i for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(tags)}
print(tag2idx)

{'O': 0, 'B-DRINKORDER': 1, 'B-PIZZAORDER': 2, 'I-DRINKORDER': 3, 'I-PIZZAORDER': 4}


In [93]:
X = [[word2idx[w[0]] for w in s] for s in sentences]
X = sequence.pad_sequences(maxlen=maxlen, sequences=X, padding="post",value=n_words - 1)

y = [[tag2idx[w[1]] for w in s] for s in sentences]
y = sequence.pad_sequences(maxlen=maxlen, sequences=y, padding="post", value=tag2idx['O'])
y = np.array([to_categorical(i, num_classes=n_tags) for i in y])
print('X shape', X.shape, 'y shape', y.shape)

X shape (15704, 22) y shape (15704, 22, 5)


In [94]:
class config():
    VOCAB = n_words
    MAX_LEN = maxlen
    N_OUPUT = n_tags
    
    
    EMBEDDING_VECTOR_LENGTH = 256
    N_LSTM_CELLS = 150
    RECURRENT_DROPOUT = 0.4
    
    OUTPUT_ACTIVATION = 'softmax'
    
    LOSS = 'categorical_crossentropy'
    OPTIMIZER = 'adam'
    METRICS = ['accuracy']
    
    MAX_EPOCHS = 5
    

In [95]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=7)
filepath = "model.keras"
ckpt = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
rlp = ReduceLROnPlateau(monitor='loss', patience=3, verbose=1)

model = Sequential()
model.add(
    Embedding(
        input_dim=config.VOCAB, output_dim=config.EMBEDDING_VECTOR_LENGTH, input_length=config.MAX_LEN
    )
)
model.add(
    Bidirectional(
        LSTM(config.N_LSTM_CELLS, return_sequences=True, recurrent_dropout=config.RECURRENT_DROPOUT)
    )
)
model.add(
    TimeDistributed(
        Dense(config.N_OUPUT, activation=config.OUTPUT_ACTIVATION)
    )
)
model.compile(loss=config.LOSS, optimizer=config.OPTIMIZER, metrics=config.METRICS)



In [96]:
history = model.fit(x=X, y=y, validation_split=0.1,
    callbacks=[es, ckpt, rlp], epochs=config.MAX_EPOCHS
)

Epoch 1/5
442/442 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.9022 - loss: 0.2805
Epoch 1: loss improved from inf to 0.10604, saving model to model.keras
442/442 ━━━━━━━━━━━━━━━━━━━━ 25s 48ms/step - accuracy: 0.9023 - loss: 0.2801 - val_accuracy: 0.9936 - val_loss: 0.0243 - learning_rate: 0.0010
Epoch 2/5
442/442 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.9942 - loss: 0.0214
Epoch 2: loss improved from 0.10604 to 0.01959, saving model to model.keras
442/442 ━━━━━━━━━━━━━━━━━━━━ 21s 48ms/step - accuracy: 0.9942 - loss: 0.0214 - val_accuracy: 0.9947 - val_loss: 0.0208 - learning_rate: 0.0010
Epoch 3/5
442/442 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.9958 - loss: 0.0160
Epoch 3: loss improved from 0.01959 to 0.01541, saving model to model.keras
442/442 ━━━━━━━━━━━━━━━━━━━━ 21s 48ms/step - accuracy: 0.9958 - loss: 0.0160 - val_accuracy: 0.9956 - val_loss: 0.0169 - learning_rate: 0.0010
Epoch 4/5
442/442 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.9963 - loss: 0.0137
Epo

In [97]:
dev_data = pd.read_json("/kaggle/input/pizza-dataset/PIZZA_dev.json", lines=True)
src_dev=dev_data["dev.SRC"].to_numpy()
top_dev=dev_data["dev.TOP"].to_numpy()


In [102]:
def evaluate(test_src,test_top):
    true_len=0

    true=0
    for src,top in zip(test_src,test_top):
        
        src_words_orginal = src.split() 
        if(len(src_words_orginal)>config.MAX_LEN):
            continue
        true_len+=1
        for word in src_words_orginal:
            if word not in words:
                str=f"(?<=\\s){word}(?=\\s)"
                src=re.sub(str,"<UNK>",src)
        src_words=src.split()
        src_indecies = [word2idx.get(w,word2idx.get("UNK", -1)) for w in src_words]
        pad_token = word2idx.get("<pad>")  
        src_indecies = src_indecies + [pad_token] * (config.MAX_LEN - len(src_words))
        src_indecies = np.expand_dims(src_indecies, axis=0)  # Shape: (1, 25)
        p = model.predict(src_indecies)  
        p = np.argmax(p, axis=-1)  # Get the predicted tag indices
        
        idx2tag = {v: k for k, v in tag2idx.items()}  
        # Iterate through predictions (p) and print the tags
        predicted=[]
        
        for i,w in enumerate(p[0]):  # p is likely a batch, so take the first sequence
            tag_begin=idx2tag[w]
            if tag_begin[0:2]=="I-" and i>0 and (predicted[-1][0:2]!="B-" or tag_begin[2:-1]!=predicted[-1][2:-1]) :
                tag_begin=re.sub("I-","B-",tag_begin)
            predicted.append(tag_begin) 
        matches = re.split(r"(?=\(PIZZAORDER|\(DRINKORDER)", top)
        start=matches[0].split()
        start=start[1:]
        matches=matches[1:]
        start_index=len(start)
        bio_tag_map={}
        for match in matches:
            match=re.sub(r'\(|\)','',match)
            match=match.strip()
            match_words= match.split()  # Split multi-word values
            tag=match_words[0]
            match_words=match_words[1:]
            for i, word in enumerate(match_words):
                if i == 1:
                    if word in src_words_orginal[start_index:]:
                        bio_tag_map[src_words_orginal.index(word,start_index)]= f"B-{tag}"  # Begin tag
                        start_index+=1
                else:
                    if word in src_words_orginal[start_index:]:
                        bio_tag_map[src_words_orginal.index(word,start_index)]= f"I-{tag}"  # Inside tag
                        start_index+=1
        word_tag_pairs = [(word, bio_tag_map.get(i, "O")) for i,word in enumerate(src_words_orginal)]
        word_tag_pairs=[wor[1] for wor in word_tag_pairs ]
        word_tag_pairs = word_tag_pairs + ["O"] * (config.MAX_LEN - len(word_tag_pairs))

        acc=[a == b for a, b in zip(word_tag_pairs, predicted)]
        if(all(acc)==True):
            true+=1
        else:
            print(src,word_tag_pairs,predicted)
        
    print("accuracy",true/true_len*100,"%")

In [103]:
evaluate(src_dev,top_dev)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
five medium pizzas with tomatoes and ham ['B-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'] ['B-PIZZAORDER', 'I-PIZZAORDER', 'O', 'O', 'B-PIZZAORDER', 'O', 'B-PIZZAORDER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
i need <UNK> <UNK> one large vegetarian pizza with extra banana peppers ['O', 'O', 'O', 'O', 'B-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'] ['O', 'O', 'B-PIZZAORDER', 'O', 'B-PIZZAORDER', 'I-PIZZAORDER', 'B-PIZZAORDER', 'O', 'O', 'B-PIZZAORDER', 'I-PIZZAORDER', 'B-PIZZAORDER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
i'd like <UNK> <UNK> a large onion and pepp

In [ ]:
def predict(sentence):
    
  
        src_words_orginal = sentence.split() 
        if(len(src_words_orginal)>config.MAX_LEN):
            return 
        for word in src_words_orginal:
            if word not in words:
                str=f"(?<=\\s){word}(?=\\s)"
                sentence=re.sub(str,"<UNK>",sentence)
        src_words=sentence.split()
        src_indecies = [word2idx.get(w,word2idx.get("UNK", -1)) for w in src_words]
        pad_token = word2idx.get("<pad>")  
        src_indecies = src_indecies + [pad_token] * (config.MAX_LEN - len(src_words))
        src_indecies = np.expand_dims(src_indecies, axis=0)  # Shape: (1, 25)
        p = model.predict(src_indecies)  
        p = np.argmax(p, axis=-1)  # Get the predicted tag indices
        
        idx2tag = {v: k for k, v in tag2idx.items()}  
        # Iterate through predictions (`p`) and print the tags
        predicted=[]
        
        for i,w in enumerate(p[0]):  # `p` is likely a batch, so take the first sequence
            predicted.append(idx2tag[w]) 
        print(sentence,predicted)
       
       
        
            

predict("I will take 2 pizza large with foul and salami")    
                
       
    

In [ ]:
# model.save("model_65.keras")